In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from google.colab import drive

# 1. Setup Paths
drive.mount('/content/drive')
BASE_DIR = "/content/drive/MyDrive/gh_voter_detector"
DATA_DIR = f"{BASE_DIR}/data"
MODEL_SAVE_PATH = f"{BASE_DIR}/final_voter_model.keras" # Saving as one complete file

# 2. Load Data (Alphabetical Order: 0='other', 1='voter_id')
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

print("Loading Training Data...")
train_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

print("Loading Validation Data...")
val_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Detected Classes: {class_names}")
# Ensure class_names outputs: ['other', 'voter_id']

# Optimize performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

# 3. Build Model (With Internal Preprocessing)
def build_model():
    # Data Augmentation (Only active during training)
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ])

    base_model = MobileNetV2(
        input_shape=IMG_SIZE + (3,),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
    x = data_augmentation(inputs)

    # CRITICAL FIX: Preprocessing is INSIDE the model
    # It expects inputs [0-255], and converts them to [-1, 1] internally.
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    return model

model = build_model()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# 4. Train
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# 5. Save the Final Model
model.save(MODEL_SAVE_PATH)
print(f"✅ Model successfully saved to: {MODEL_SAVE_PATH}")